In [7]:
import sqlite3
from zipfile import ZipFile
import pandas as pd
import json
import time

In [3]:
#Создаем базу данных
conn = sqlite3.connect('hw1.db')

In [4]:
cur = conn.cursor()

In [5]:
#создаем пустую таблицу для справочника оквед
cur.execute("CREATE TABLE okved (code text, parent_code text, section text, name text, comment text);")


In [15]:
#создаем пустую таблицу для данных по юр. лицам
cur.execute("CREATE TABLE egrul (ogrn text, inn text, kpp text, name text, КодОКВЭД text);")

In [6]:
# Достаем из архива справочник и загружаем в sql
with ZipFile("C:/Users/vitaliy.y.karpov/Desktop/Продвинутый python/ДЗ1/okved_2.json.zip", "r") as zf:
        for file in zf.infolist():
            okved_2 = zf.read(file)
            okved_2 = okved_2.decode('utf-8')
            text = json.loads(json.dumps(okved_2))
            okved = pd.read_json(text, dtype=str)
            okved.to_sql('okved', conn, if_exists='append', index = bool)


In [16]:
tic = time.perf_counter()
#распаковываем архив с данными по юр. лицам
with ZipFile("C:/Users/vitaliy.y.karpov/Desktop/Продвинутый python/ДЗ1/egrul.json.zip", "r") as zf:
    file_names = zf.namelist()
    for name in file_names:
        data = json.loads(zf.read(name).decode())
        extracted = pd.json_normalize(data)
        #делаем проверку на наличие столбцов
        try:
            extracted = extracted[['ogrn','inn','kpp','name','data.СвОКВЭД.СвОКВЭДОсн.КодОКВЭД']]
        except KeyError:
            continue
        extracted.rename(columns = {'data.СвОКВЭД.СвОКВЭДОсн.КодОКВЭД':'КодОКВЭД'}, inplace = True)
        extracted = extracted.dropna()
        #создаем отфильтрованную таблицу для загрузки в бд
        egrul = extracted[extracted.КодОКВЭД.str.startswith('61')]
        egrul.to_sql('egrul', conn, if_exists='append', index = bool)
        
toc = time.perf_counter()
print(f"Вычисление заняло {toc - tic:0.4f} секунд")   

#время на отработку скрипта ушло 2 часа 13 мин.
#Загружено 20917 строк

Вычисление заняло 7679.8304 секунд


,ogrn,inn,kpp,name,КодОКВЭД
722,1037861007799,7838003370,470501001,"ООО ""РЭЙС СЕВЕРО-ЗАПАД""",61.10.1


In [17]:
conn.close()